In [1]:
hf_tokens = "API_KEY"
mistal_token = "MfEJCUufH1MbNzi1CbeenN3YPXSse4bC"

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

In [3]:
!pip install -q mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.3/216.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [4]:

import os
from mistralai import Mistral
import re


client = Mistral(api_key=mistal_token)

# Agents

You can create an Agent in https://console.mistral.ai/build/agents/new, for this notebook we will use mistral-large-2407 as the model powering our agents!

## Don't use Few-shot prompting : it doesn't work





---

### **Planning Agent Prompt:**

You are a data analytical planning assistant. Given a dataset containing "profit-loss", "balance-sheet", "cash-flow", "quarters" (stored inside the "report_type" column of a single dataframe) of Nifty50 stocks over a 12-year period, your task is to provide specific and simple analysis plans that apply a modified GARP (Growth at a Reasonable Price) strategy to find the 5 best stocks to invest in for the next 5 years.

The modified GARP strategy involves the following criteria:
- **Market Capitalization:** More than ₹25,000 crore
- **Sales:** Sales > ₹20,000 crore
- **PEG Ratio:** 0 < PEG Ratio < 1.5
- **PE Ratio:** PE Ratio < 30
- **Sales Growth:**
  - Year-over-year (YoY) sales growth > 15%
  - Sales growth over the last 3 years > 15%
  - Sales growth over the last 5 years > 15%
- **EPS Growth:**
  - Current year EPS > 1.15 times the last year's EPS
  - EPS growth over the last 3 years > 15%
  - EPS growth over the last 5 years > 15%
- **Expected Quarterly Net Profit:** Expected quarterly net profit should be greater than the net profit of 3 quarters ago.

Provide an analysis plan and instructions for implementing this strategy.

Return output with the following format:

- **Total number of steps:**
  
- **Step 1:**
  Describe the overall process to clean and prepare the dataset for analysis, including handling missing values and ensuring consistency across the financial statements.

- **Step 2:**
  Explain how to calculate the key financial metrics relevant to the modified GARP strategy, such as market capitalization, sales growth, PEG ratio, EPS growth, and expected quarterly net profit.

- **Step 3:**
  Describe how to filter and screen the stocks based on the modified GARP criteria (e.g., market cap > ₹25,000 crore, sales > ₹20,000 crore, PEG ratio between 0 and 1.5, EPS growth > 15%, etc.).

- **Step 4:**
  Provide instructions on how to rank the stocks based on these criteria to identify the top 5 stocks with the best growth potential at a reasonable price.

- **Step 5:**
  Detail the plan for summarizing the analysis results in a structured format to be provided to the Summarization Agent.

---

### **Python Agent Prompt:**

You are a Python coding assistant that only outputs Python code with explanations or comments. Given the following instructions from the Planning Agent, you need to generate Python code for analyzing a dataset containing 12 years of financial data for Nifty50 stocks, including "profit-loss", "balance-sheet", "cash-flow", "quarters" (stored inside "report_type" column of a single dataframe).

You will:
1. Clean and prepare the dataset by handling missing values and ensuring consistency across the data.
2. Calculate key financial metrics as per the modified GARP strategy:
   - Market capitalization
   - Sales (YoY and over 3 and 5 years)
   - PEG ratio
   - PE Ratio
   - EPS growth (YoY and over 3 and 5 years)
   - Expected quarterly net profit (greater than the net profit from 3 quarters back).
3. Screen the stocks based on the modified GARP criteria (e.g., market cap > ₹25,000 crore, sales > ₹20,000 crore, PEG ratio between 0 and 1.5, EPS growth > 15%, etc.).
4. Rank the stocks and identify the top 5 based on these metrics.

Ensure that your code is well-documented, modular, and efficient, with comments explaining each step. Return only the Python code.

---

### **Summarization Agent Prompt:**

You are an analysis summarization assistant. You have access to a dataset containing 12 years of financial data for Nifty50 stocks ("profit-loss", "balance-sheet", "cash-flow", "quarters" - all stored inside the "report_type" column of a single dataframe) and the results of an analysis that applied a modified GARP (Growth at a Reasonable Price) strategy to identify the top 5 stocks for investment.

The modified GARP strategy includes the following criteria:
- Market Capitalization > ₹25,000 crore
- Sales > ₹20,000 crore
- PEG Ratio between 0 and 1.5
- PE Ratio less than 30
- YoY Sales Growth > 15%, Sales growth over the last 3 years > 15%, and Sales growth over the last 5 years > 15%
- EPS growth > 15% YoY, over 3 years, and over 5 years
- Expected quarterly net profit > net profit from 3 quarters ago.

Your task is to generate a concise analysis report summarizing:
1. The financial health and growth potential of the selected 5 stocks.
2. Key criteria used, including market cap, sales growth, PEG ratio, PE ratio, and EPS growth.
3. Any notable trends or observations from the 12 years of financial data.
4. The rationale behind the selection of each stock as a top pick for investment over the next 5 years.

The report should be structured as follows:
- **Introduction:** Brief overview of the dataset and the modified GARP strategy.
- **Methodology:** Summary of the analysis process (how the key metrics were calculated and how stocks were screened).
- **Results:** Summary of the top 5 stocks, with their actual `stock_symbol` (from the dataframe) and a brief explanation of why each stock was chosen.
- **Conclusion:** Final remarks on the analysis and investment outlook based on the findings. Provide the actual name of the selected stocks (`stock_symbol` mentioned in the dataframe) according to your analysis.





In [5]:
planning_agent_id = "ag:ce02e123:20240908:planning-agent:e23e7010"
python_agent_id = "ag:ce02e123:20240908:python-agent:99e4e80c"
summarization_agent_id = "ag:ce02e123:20240908:summarization-agent-:3021ca3c"

# Analysis Planning

In [6]:
import requests
import json

def run_analysis_planning_agent(query):
    """
    Sends a user query to a Python agent and returns the response.

    Args:
        query (str): The user query to be sent to the Python agent.

    Returns:
        str: The response content from the Python agent.
    """
    print("### Run Planning agent")
    print(f"User query: {query}")
    try:
        response = client.agents.complete(
            agent_id= planning_agent_id,
            messages = [
                {
                    "role": "user",
                    "content":  query
                },
            ]
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

In [7]:
query = """
Load this data: https://github.com/Pritam3355/Datasets/blob/main/fundamental.csv

The dataset consists of 21866 datapoints and has six columns:
- category : type of financial metrics like Expenses_profit-loss,Equity Capital,Net Cash Flow,Sales_quarters
- report_type : can be any one of "profit-loss", "balance-sheet", "cash-flow", "quarters"
- stock_symbol : contains the name of the stock
- year : year of the report
- month : month of the report
- value_type : value of category of that particular stock
"""

In [8]:
planning_result = run_analysis_planning_agent(query)

### Run Planning agent
User query: 
Load this data: https://github.com/Pritam3355/Datasets/blob/main/fundamental.csv

The dataset consists of 21866 datapoints and has six columns:
- category : type of financial metrics like Expenses_profit-loss,Equity Capital,Net Cash Flow,Sales_quarters
- report_type : can be any one of "profit-loss", "balance-sheet", "cash-flow", "quarters"
- stock_symbol : contains the name of the stock
- year : year of the report
- month : month of the report
- value_type : value of category of that particular stock



In [9]:
print(planning_result)

### Total number of steps: 5

---

### Step 1: Clean and Prepare the Dataset

**Instructions:**

1. **Load the Dataset:**
   - Load the dataset from the provided URL into a dataframe.

2. **Handle Missing Values:**
   - Check for missing values in the dataset.
   - Decide on an appropriate strategy for handling missing values (e.g., filling with mean/median, forward/backward fill, or dropping rows/columns with missing values).

3. **Ensure Consistency:**
   - Verify that the financial statements (profit-loss, balance-sheet, cash-flow, quarters) are consistent across different stocks and time periods.
   - Standardize the format of the "value_type" column to ensure consistent data types (e.g., converting all values to numeric).

4. **Create Additional Columns:**
   - Create a new column for "quarter" by combining the "year" and "month" columns.
   - Ensure that the "stock_symbol" column is in a consistent format.

---

### Step 2: Calculate Key Financial Metrics

**Instructions:**

1. *

# Generate and execute Python code for each planning step

In [10]:
class PythonAgentWorkflow:
    def __init__(self):
        pass

    def extract_pattern(self, text, pattern):
        """
        Extracts a pattern from the given text.

        Args:
            text (str): The text to search within.
            pattern (str): The regex pattern to search for.

        Returns:
            str: The extracted pattern or None if not found.
        """
        match = re.search(pattern, text, flags=re.DOTALL)
        if match:
            return match.group(1).strip()
        return None

    def extract_step_i(self, planning_result, i, n_step):
        """
        Extracts the content of a specific step from the planning result.

        Args:
            planning_result (str): The planning result text.
            i (int): The step number to extract.
            n_step (int): The total number of steps.

        Returns:
            str: The extracted step content or None if not found.
        """
        if i < n_step:
            pattern = rf'## Step {i}:(.*?)## Step {i+1}'
        elif i == n_step:
            pattern = rf'## Step {i}:(.*)'
        else:
            print(f"Invalid step number {i}. It should be between 1 and {n_step}.")
            return None

        step_i = self.extract_pattern(planning_result, pattern)
        if not step_i:
            print(f"Failed to extract Step {i} content.")
            return None

        return step_i

    def extract_code(self, python_agent_result):
          """
          Extracts Python function and test case from the response content.

          Args:
              result (str): The response content from the Python agent.

          Returns:
              tuple: A tuple containing the extracted Python function and a retry flag.
          """
          retry = False
          # # Check if python_agent_result is None. If so, set retry to True and return None, True
          # if python_agent_result is None:
          #     retry = True
          #     return None, retry
          print("### Extracting Python code")
          python_code = self.extract_pattern(python_agent_result, r'```python(.*?)```')
          if not python_code:
              retry = True
              print("Python function failed to generate or wrong output format. Setting retry to True.")

          return python_code, retry

    def run_python_agent(self, query):
        """
        Sends a user query to a Python agent and returns the response.

        Args:
            query (str): The user query to be sent to the Python agent.

        Returns:
            str: The response content from the Python agent.
        """
        print("### Run Python agent")
        print(f"User query: {query}")
        try:
            response = client.agents.complete(
                agent_id= python_agent_id,
                messages = [
                    {
                        "role": "user",
                        "content":  query
                    },
                ]
            )
            result = response.choices[0].message.content
            return result

        except Exception as e:
            print(f"Request failed: {e}. Please check your request.")
            return None

    def check_code(self, python_function, state):
        """
        Executes the Python function and checks for any errors.

        Args:
            python_function (str): The Python function to be executed.

        Returns:
            bool: A flag indicating whether the code execution needs to be retried.

        Warning:
            This code is designed to run code that’s been generated by a model, which may not be entirely reliable.
            It's strongly recommended to run this in a sandbox environment.
        """
        retry = False
        try:
            print(f"### Python function to run: {python_function}")
            exec(python_function, state)
            print("Code executed successfully.")
        except Exception:
            print(f"Code failed.")
            retry = True
            print("Setting retry to True")
        return retry

    def process_step(self, planning_result, i, n_step, max_retries, state):
        """
        Processes a single step, including retries.

        Args:
            planning_result (str): The planning result text.
            i (int): The step number to process.
            n_step (int): The total number of steps.
            max_retries (int): The maximum number of retries.

        Returns:
            str: The extracted step content or None if not found.
        """

        retry = True
        j = 0
        while j < max_retries and retry:
            print(f"TRY # {j}")
            j += 1
            step_i = self.extract_step_i(planning_result, i, n_step)
            if step_i:
                print(step_i)
                python_agent_result = self.run_python_agent(step_i)
                python_code, retry = self.extract_code(python_agent_result)
                print(python_code)
                retry = self.check_code(python_code, state)
        return None

    def workflow(self, planning_result):
        """
        Executes the workflow for processing planning results.

        Args:
            planning_result (str): The planning result text.
        """
        state = {}
        print("### ENTER WORKFLOW")
        n_step = int(self.extract_pattern(planning_result, '## Total number of steps:\s*(\d+)'))
        for i in range(1, n_step + 1):
            print(f"STEP # {i}")
            self.process_step(planning_result, i, n_step, max_retries=2, state=state)


        print("### Exit WORKFLOW")
        return None


In [11]:
import sys
import io

# See the output of print statements in the console while also capturing it in a variable,
class Tee(io.StringIO):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.original_stdout = sys.stdout

    def write(self, data):
        self.original_stdout.write(data)
        super().write(data)

    def flush(self):
        self.original_stdout.flush()
        super().flush()

# Create an instance of the Tee class
tee_stream = Tee()

# Redirect stdout to the Tee instance
sys.stdout = tee_stream


Python_agent = PythonAgentWorkflow()
Python_agent.workflow(planning_result)

# Restore the original stdout
sys.stdout = tee_stream.original_stdout

# Get the captured output
captured_output = tee_stream.getvalue()

### ENTER WORKFLOW
STEP # 1
TRY # 0
Clean and Prepare the Dataset

**Instructions:**

1. **Load the Dataset:**
   - Load the dataset from the provided URL into a dataframe.

2. **Handle Missing Values:**
   - Check for missing values in the dataset.
   - Decide on an appropriate strategy for handling missing values (e.g., filling with mean/median, forward/backward fill, or dropping rows/columns with missing values).

3. **Ensure Consistency:**
   - Verify that the financial statements (profit-loss, balance-sheet, cash-flow, quarters) are consistent across different stocks and time periods.
   - Standardize the format of the "value_type" column to ensure consistent data types (e.g., converting all values to numeric).

4. **Create Additional Columns:**
   - Create a new column for "quarter" by combining the "year" and "month" columns.
   - Ensure that the "stock_symbol" column is in a consistent format.

---

#
### Run Python agent
User query: Clean and Prepare the Dataset

**Instruction

# Summarization

In [12]:
response = client.agents.complete(
    agent_id= summarization_agent_id,
    messages = [
        {
            "role": "user",
            "content":  query + captured_output
        },
    ]
)
result = response.choices[0].message.content


In [13]:
print(result)

### Analysis Summarization Report

#### Introduction:
This report provides a comprehensive analysis of a dataset containing 12 years of financial data for Nifty50 stocks. The analysis employs a modified GARP (Growth at a Reasonable Price) strategy to identify the top 5 stocks for investment. The dataset includes financial metrics such as "profit-loss", "balance-sheet", "cash-flow", and "quarters".

#### Methodology:
The analysis involves several key steps:
1. **Data Cleaning and Preparation:**
   - Handle missing values by filling them with appropriate methods.
   - Ensure consistency across the dataset, including converting columns to appropriate data types.
   - Create additional columns for "quarter" by combining "year" and "month" columns and ensuring consistent formatting of "stock_symbol".

2. **Calculation of Key Financial Metrics:**
   - Market Capitalization: Calculated as `Market Cap = Stock Price * Number of Shares Outstanding`.
   - Sales Growth: Calculated year-over-year (